🌍 Project: Automated ETL Pipeline for Global GDP Data
📌 Project Overview
Developed an automated ETL (Extract, Transform, Load) pipeline to retrieve, process, and store global GDP data as reported by the International Monetary Fund (IMF).

🎯 Objectives
Extract: Scrape real-time GDP data from the Wikipedia archive using BeautifulSoup.

Transform: Convert GDP values from 'Million USD' to 'Billion USD' and round to 2 decimal places.

Load: Save the processed data into a CSV file and an SQLite database.

Verify: Execute SQL queries to filter countries with an economy larger than $100 Billion.

Audit: Maintain an execution log with precise timestamps for process monitoring.

🛠️ Technical Stack
Language: Python

Libraries: pandas, requests, BeautifulSoup, sqlite3

Database: SQLite3

📂 Key Outputs
Countries_by_GDP.csv: Structured CSV report of processed data.

World_Economies.db: SQL database containing the Countries_by_GDP table.

etl_project_log.txt: Complete audit trail of the pipeline execution.

🚀 Usage
Bash
    python3 etl_project_gdp.py
Next Step:
Would you like me to help you write a professional GitHub README.md file including a "How to Install" section for this project?

In [1]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

db_name= './World_Economies.db'
table_name= 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'


def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./etl_project_log.txt","a") as f:
        f.write(timestamp + ' : ' + message + '\n')


def myExtract():
    url='https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
    try:
        response = requests.get(url,timeout=60)
        response.raise_for_status()
        html_page = response.text
    except Exception as e: 
        log_progress(f"Extraction failed: {str(e)}")
        print(f"Error: {e}")
        return pd.DataFrame() 

    data = BeautifulSoup(html_page, 'html.parser')

    # Create Empty Dataframe
    df=pd.DataFrame(columns=["Country","GDP_USD_million"])
    tables= data.find_all('tbody')

    rows= tables[2].find_all('tr')

    for row in rows:
        col = row.find_all('td')
        if len(col) !=0:
            if col[0].find('a') is not None and '-' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_million": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict,index=[0])
                df= pd.concat([df,df1], ignore_index=True)
    return df

def transform(df):
    print("Started transform stage.")
    #convert string to digit and divided by 1000
    gdp_list = df["GDP_USD_million"].tolist()
    new_gdp_list=[]

    for x in gdp_list:
        if x == '—' or x == '-' or x == '' or x is None:
            new_gdp_list.append(0.0)
        else:
            clean_value = float("".join(str(x).split(',')))
            new_gdp_list.append(round(clean_value / 1000, 2))

    df["GDP_USD_million"] = new_gdp_list
    df = df.rename(columns={"GDP_USD_million":"GDP_USD_billion"})
    return df

def load_to_csv(df, csv_path):
    df.to_csv(csv_path)
    
def load_to_db(df,sql_connection,table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    log_progress('Data loaded to Database as table.')

log_progress('Preliminaries complete. Initiating ETL process')

df=myExtract()
log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)
log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated')
load_to_db(df, sql_connection, table_name)
query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billion >= 100"
query_output = pd.read_sql(query_statement, sql_connection)
print("\nCountries with GDP > 100 Billion USD:")
print(query_output)
log_progress('Query executed. Process Complete.')
sql_connection.close()

Started transform stage.

Countries with GDP > 100 Billion USD:
          Country  GDP_USD_billion
0   United States         26854.60
1           China         19373.59
2           Japan          4409.74
3         Germany          4308.85
4           India          3736.88
..            ...              ...
64          Kenya           118.13
65         Angola           117.88
66           Oman           104.90
67      Guatemala           102.31
68       Bulgaria           100.64

[69 rows x 2 columns]
